In [11]:
import os, warnings, sys
import pandas as pd
import pyarrow.parquet as pq
import dask.dataframe as dd

sys.path.insert(1, 'C:/Users/crist/GitHub/work/gestao/Library/mongodb')

import MongoUtils

warnings.filterwarnings('ignore')

In [12]:
path = 'D:/Disco/Data/datasus'


In [17]:
datasus = MongoUtils.MongoUtils()
datasus.connect('mongodb://localhost:27017', 'datasus')
datasus.initCollection('sim')
datasus.initCollection('sinan')
datasus.initCollection('sinasc')


In [15]:
sim = pq.ParquetDataset(f'{path}/SIM', use_legacy_dataset=False)
sinan = pq.ParquetDataset(f'{path}/SINAN', validate_schema=False)
sinasc = pq.ParquetDataset(f'{path}/SINASC', use_legacy_dataset=False)

In [ ]:
# le 20% do total de registros para criar um dataset exploratorio
df_sim = sim.read().to_pandas().sample(frac=0.20)
df_sim.to_csv(f'{path}/SIM_20.csv', index=False)
del df_sim

In [ ]:
df_sinasc = sinasc.read().to_pandas().sample(frac=0.20)
df_sinasc.to_csv(f'{path}/SINASC_20.csv', index=False)
del df_sinasc

In [ ]:
df_sim = pd.read_csv(f'{path}/SIM_20.csv')
#df_sinasc = pd.read_csv(f'{path}/SINASC_20.csv')

In [16]:
def insert_to_mongo(file, collection):
    print(file)
    records = pd.read_parquet(file).to_dict('records')
    datasus.insert_many(collection, records)

In [18]:
for file in sim.files:
    insert_to_mongo(file, 'sim')


D:/Disco/Data/datasus/SIM/SIM_AC_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_AL_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_AM_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_AP_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_BA_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_CE_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_DF_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_ES_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_GO_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_MA_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_MG_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_MS_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_MT_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_PA_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_PB_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_PE_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_PI_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_PR_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_RJ_1994-2021.parquet
D:/Disco/Data/datasus/SIM/SIM_R

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:

for file in sinan.files:
    insert_to_mongo(file, 'sinan')

for file in sinasc.files:
    insert_to_mongo(file, 'sinasc')

In [ ]:
#transforma um conjunto de arquivos em apenas um arquivo
data = pd.DataFrame()

for file in os.listdir(f'{path}'):
    data = data.append(pd.read_parquet(f'{path}{file}'))

data.to_parquet(f'{path}SINASC_BR_1994-2021.parquet')

In [ ]:
# dados adicionais
ocupacoes = pd.read_csv('https://raw.githubusercontent.com/daltonbc96/python/master/CBO.csv')
municipios = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv')
#pd.read_csv('http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv', sep=';', encoding='latin-1')

In [ ]:
df_sim[df_sim[['DATANASC']].notna()]

In [ ]:
df_sim = sim.read().to_pandas()#.sample(frac=0.20)
#df_sim.to_csv(f'{path}/SIM_20.csv', index=False)
#del df_sim

In [ ]:
df_sim_rs = df_sim[df_sim['STATE'] == 'RS']

In [ ]:
df_sim_rs.to_csv(f'{path}/SIM_RS.csv', index=False)

In [ ]:
df_sim_rs = df_sim_rs[['DATAREG', 'DATAOBITO', 'DATANASC', 'IDADE', 'TIPOBITO', 'ESTCIVIL', 'SEXO', 'RACACOR', 'CODMUNRES', 'LOCOCOR', 'CODMUNOCOR', 'OCUP', 'CODMUNNATU', ]]


In [ ]:
# sava dados do SIM
for year in range(1993, 2022):
    print(year)
    data = pd.read_csv(f'https://diaad.s3.sa-east-1.amazonaws.com/sim/Mortalidade_Geral_{year}.csv', sep=';', encoding='latin-1')
    data.to_csv(f'{path}/SIM/SIM_{year}.csv', index=False)

    data = data.to_dict('records')
    datasus.insert_many('sim', data)
    print('ok')
